In [ ]:
import os
import pandas as pd
from dotenv import load_dotenv
from modules.postgres_functions import probar_conexion_postgresql, create_sql_script, sql_query, df_to_sql

def load_env():
    load_dotenv()
    host = os.getenv("HOST")
    port = os.getenv("PORT")
    database = os.getenv("DATABASE")
    user = os.getenv("USER")
    password = os.getenv("PASSWORD")
    return host, port, database, user, password

host, port, database, user, password = load_env()
probar_conexion_postgresql(host, password, database, user, port)

In [3]:
excel_path = "Table.xlsx"
df1 = pd.read_excel(excel_path, sheet_name="Servicios")
df2 = pd.read_excel(excel_path, sheet_name="Etapas")
df3 = pd.read_excel(excel_path, sheet_name="Informes")

In [4]:
def create_first_table(df, table_name, file_name, pk_col=None, index_cols=None, date_cols=False):
    create_sql_script(df, table_name, file_name, pk_name=pk_col, index_cols=index_cols, date_cols=date_cols)

def create_sql_table(df, table_name, file_name, pk_col=None, index_cols=None, date_cols=False, final_table=None):
    create_sql_script(df, table_name, file_name, pk_name=pk_col, index_cols=index_cols, date_cols=date_cols, transaction=final_table)
    with open(file_name, 'r') as file:
        query = file.read()
    sql_query(host, password, database, user, port, query, fetch=False)
    df_to_sql(host, password, database, user, port, df, table_name, 'append', index_label=None)


In [ ]:
file_name = 'script_table.sql'
dir = os.path.join(os.getcwd(), 'lib')
if not os.path.exists(dir):
    os.makedirs(dir)

file_name = os.path.join(dir, file_name)

pk_col = 'ID UNICO'
index_cols = ['Ano vigente', 'ID tipologia']

create_sql_script(df, 'tabla_temporal', file_name, index_cols=index_cols, date_cols=True, pk_name=pk_col, transaction='tabla')

with open(file_name, 'r') as file:
    query = file.read()
sql_query(host, password, database, user, port, query, fetch=False)

df_to_sql(host, password, database, user, port, df, 'tabla_temporal', 'append', index_label=None)